# Scene Understanding

This is very crucial need for autonimous vehicles. Instead of just indentifying a class, what we is expected is see the whole secene as humans and indetify every class like people, vehciles, signs etc with high pixel precision where we know pixel by pixle where these classes end from other objects in scene.

## 1. Our first task to detect objects. 

**Bounding Boxes** has been one of the great methods to classify objects. YOLO and SSD models are great open source and state of the art solutions which can detect people, cars etc. However, bounding boxes have limitations, imagine classifying these in curvy road, the forest or sky. This quickly becomes problematic or even impossible covey true shape of the object. At best bounding boxes can help partial scene understanding. 

**Semantic Segmentation** is the task of assigning meaning to part of an object, this can be done at pixel level, where we can assign every pixel to target class example road, vehicle, people, signs etc. This helps look at every pixel in the scene instead of slicing sections into bounding boxes.

This is highly relevant to the autonomous vehicle to effectively make decisions. 

### One approach to scene understanding is to train multiple decoders.

Each decoder is assigned with seperate task, example:
- One decoder can do segmentation
- and another for depth measurement

This way we can have single network which not only predicts class of the pixel but additionally how far away it is. This can help construct rich 3D scenes like how we humans do. This takes us to strong visual perception and intution however we will keep things simple for now .. 

### IOU : Intersection over Union Metrics, commonly used to measure performance of a model on the semantic segmentation task

It is literally : Intersection Set / Union Set

Intersection Set (AND): It is intersection of two sets, that is AND operation, if object exists in both sets, it will be parked under intersection set.

Intersection is defined as number of pixels that are both truly part of that class & are classified as part of the class by the network.

Union Set (OR): Union of the two set is a OR operation. If it exists in at least one of the two , then we put in the union set.

The union is defined as number of pixels that are truly part of that class plus the nuber of pixels that are classified as part of that class by the network.

So the Intersection Set (AND) <= Union Set (OR).

The ratio then tell us the overall performance per pixel, per class.

And this ratio: Intersection Set / Union Set <= 1

We can even go further & calculate mean IOU for a network, which is just the avergae of all the IOU for all the classes. This gives you an idea of how well it handles all te different classifications for every single pixel.

In tensorflow we can use mean IOU function.

In [1]:
import tensorflow as tf


def mean_iou(ground_truth, prediction, num_classes):
    # TODO: Use `tf.metrics.mean_iou` to compute the mean IoU.
    iou, iou_op = tf.metrics.mean_iou(ground_truth, prediction, num_classes)
    return iou, iou_op


ground_truth = tf.constant([
    [0, 0, 0, 0], 
    [1, 1, 1, 1], 
    [2, 2, 2, 2], 
    [3, 3, 3, 3]], dtype=tf.float32)
prediction = tf.constant([
    [0, 0, 0, 0], 
    [1, 0, 0, 1], 
    [1, 2, 2, 1], 
    [3, 3, 0, 3]], dtype=tf.float32)
    
# TODO: use `mean_iou` to compute the mean IoU
iou, iou_op = mean_iou(ground_truth, prediction, 4)

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # need to initialize local variables for this to run `tf.metrics.mean_iou`
        sess.run(tf.local_variables_initializer())
        
        sess.run(iou_op)
        # should be 0.53869
        print("Mean IoU =", sess.run(iou))

Mean IoU = 0.53869


### Let's see how to technically implement Encoders and Decoders

**FCN-8 - Encoder**

Let’s focus on a concrete implementation of a fully convolutional network. We’ll discuss the FCN-8 architecture developed at Berkeley. In fact, many FCN models are derived from this FCN-8 implementation. The encoder for FCN-8 is the VGG16 model pretrained on ImageNet for classification. The fully-connected layers are replaced by 1-by-1 convolutions. Here’s an example of going from a fully-connected layer to a 1-by-1 convolution in 

TensorFlow:

num_classes = 2
output = tf.layers.dense(input, num_classes)

To:

num_classes = 2
output = tf.layers.conv2d(input, num_classes, 1, strides=(1,1))

The third argument, 1, is the kernel size, meaning this is a 1 by 1 convolution. Thus far, we’ve downsampled the input image and extracted features using the VGG16 encoder. We’ve also replaced the linear layers with 1 by 1 convolutional layers, preserving spatial information.

But this is just the encoder portion of the network. Next comes the decoder.

**FCN-8 - Decoder**
To build the decoder portion of FCN-8, we’ll upsample the input to the original image size. The shape of the tensor after the final convolutional transpose layer will be 4-dimensional: (batch_size, original_height, original_width, num_classes). Let’s implement those transposed convolutions we discussed earlier as follows:

output = tf.layers.conv2d_transpose(input, num_classes, 4, strides=(2, 2))

The transpose convolutional layers increase the height and width dimensions of the 4D input Tensor.

**Skip Connections**
The final step is adding skip connections to the model. In order to do this we’ll combine the output of two layers. The first output is the output of the current layer. The second output is the output of a layer further back in the network, typically a pooling layer. In the following example we combine the result of the previous layer with the result of the 4th pooling layer through elementwise addition (tf.add).

-## make sure the shapes are the same!
input = tf.add(input, pool_4)

We can then follow this with another transposed convolution layer.

input = tf.layers.conv2d_transpose(input, num_classes, 4, strides=(2, 2))
We’ll repeat this once more with the third pooling layer output.

input = tf.add(input, pool_3)
Input = tf.layers.conv2d_transpose(input, num_classes, 16, strides=(8, 8))

**FCN-8 - Classification & Loss**
The final step is to define a loss. That way, we can approach training a FCN just like we would approach training a normal classification CNN.

In the case of a FCN, the goal is to assign each pixel to the appropriate class. We already happen to know a great loss function for this setup, cross entropy loss! Remember the output tensor is 4D so we have to reshape it to 2D:

...
logits = tf.reshape(input, (-1, num_classes))

logits is now a 2D tensor where each row represents a pixel and each column a class. From here we can just use standard cross entropy loss:

cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))

That’s it, we now have an end-to-end model for semantic segmentation. Time to get training!